In [6]:
'''B-1.モデルの学習
https://qiita.com/tshimba/items/60f7bcc8ae26b5004a3a#_reference-5d15f52415912c9c2182

上記を参考にしたバージョン①
'''

'''

'''

import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import TensorBoard
import numpy as np
from keras.callbacks import TensorBoard
#from smallcnn import save_history

def save_history(history, result_file):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(result_file, "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))

classes = ['dogs', 'cats']

batch_size = 50
nb_classes = len(classes)

img_rows, img_cols = 224, 224
channels = 3

train_data_dir = '/Users/yoshimurakeita/Desktop/work/20180527/dataset/trainData/'
validation_data_dir = '/Users/yoshimurakeita/Desktop/work/20180527/dataset/validationData/'
log_dir = '/Users/yoshimurakeita/Desktop/work/20180527/tensorboard'

nb_train_samples = 2000
nb_val_samples = 800
nb_epoch = 100

result_dir = '/Users/yoshimurakeita/Desktop/work/20180527/results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

    
# callbacks
callbacks = TensorBoard(log_dir=log_dir,
                                        histogram_freq=0)


# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
# vgg16.summary()


# FC層を構築
# Flattenへの入力指定はバッチ数を除く
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))


# VGG16とFCを接続
model = Model(input=vgg16.input, 
              output=top_model(vgg16.output))


# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:15]:
    layer.trainable = False


# Fine-tuningのときはSGDの方がよい？
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


# generator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)


/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
'''B-2.fit
https://qiita.com/tshimba/items/60f7bcc8ae26b5004a3a#_reference-5d15f52415912c9c2182

上記を参考にしたバージョン②
'''

# Fine-tuning
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=nb_train_samples / batch_size,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    callbacks=[callbacks],  # tensorboardで処理内容を確認する(yoshimが追加しました)
    validation_steps=None)

/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(steps_per_epoch=40.0, validation_steps=None, generator=<keras.pre..., callbacks=[<keras.ca..., epochs=100, validation_data=<keras.pre...)`
  


Epoch 1/100
40/40 [==============================] - 551s 14s/step - loss: 0.7487 - acc: 0.5580 - val_loss: 0.5687 - val_acc: 0.7300
Epoch 2/100
40/40 [==============================] - 555s 14s/step - loss: 0.5720 - acc: 0.7090 - val_loss: 0.4774 - val_acc: 0.8075
Epoch 3/100
40/40 [==============================] - 556s 14s/step - loss: 0.5000 - acc: 0.7585 - val_loss: 0.4071 - val_acc: 0.8350
Epoch 4/100
40/40 [==============================] - 557s 14s/step - loss: 0.4355 - acc: 0.8055 - val_loss: 0.3597 - val_acc: 0.8575
Epoch 5/100
40/40 [==============================] - 564s 14s/step - loss: 0.3958 - acc: 0.8225 - val_loss: 0.3254 - val_acc: 0.8738
Epoch 6/100
40/40 [==============================] - 560s 14s/step - loss: 0.3688 - acc: 0.8470 - val_loss: 0.3020 - val_acc: 0.8800
Epoch 7/100
40/40 [==============================] - 561s 14s/step - loss: 0.3295 - acc: 0.8610 - val_loss: 0.2843 - val_acc: 0.8813
Epoch 8/100
40/40 [==============================] - 576s 14s/step - 

40/40 [==============================] - 560s 14s/step - loss: 0.0780 - acc: 0.9720 - val_loss: 0.1678 - val_acc: 0.9325
Epoch 63/100
40/40 [==============================] - 560s 14s/step - loss: 0.0764 - acc: 0.9705 - val_loss: 0.1695 - val_acc: 0.9362
Epoch 64/100
40/40 [==============================] - 561s 14s/step - loss: 0.0745 - acc: 0.9710 - val_loss: 0.1756 - val_acc: 0.9325
Epoch 65/100
40/40 [==============================] - 560s 14s/step - loss: 0.0726 - acc: 0.9745 - val_loss: 0.1686 - val_acc: 0.9337
Epoch 66/100
40/40 [==============================] - 560s 14s/step - loss: 0.0773 - acc: 0.9690 - val_loss: 0.1692 - val_acc: 0.9375
Epoch 67/100
40/40 [==============================] - 560s 14s/step - loss: 0.0774 - acc: 0.9760 - val_loss: 0.1670 - val_acc: 0.9363
Epoch 68/100
40/40 [==============================] - 561s 14s/step - loss: 0.0622 - acc: 0.9785 - val_loss: 0.1696 - val_acc: 0.9362
Epoch 69/100
40/40 [==============================] - 560s 14s/step - loss:

In [8]:
'''B-3.パラメータの保存
https://qiita.com/tshimba/items/60f7bcc8ae26b5004a3a#_reference-5d15f52415912c9c2182

上記を参考にしたバージョン②
'''

model.save_weights(os.path.join(result_dir, 'finetuning.h5'))
save_history(history, os.path.join(result_dir, 'history_finetuning.txt'))

In [ ]:
%%bash

tensorboard --logdir=/Users/yoshimurakeita/Desktop/work/20180527/tensorBoard/


'''
なぜか上記のシェルが動かなかったので、ターミナルから上記を起動すると、URLが表示されるのでそのURLにアクセスする。
'''


In [5]:
'''B-4.fit

B-2の続きからさらに学習する際は、下記の要領でやる
'''

nb_epoch = 50

# Fine-tuning
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=nb_train_samples / batch_size,
    nb_epoch=nb_epoch,
    validation_data=validation_generator,
    callbacks=[callbacks],  # tensorboardで処理内容を確認する(yoshimが追加しました)
    validation_steps=None,
    initial_epoch=10) # ここで、上記の最後のエポックを入れればOK

/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  app.launch_new_instance()
/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(steps_per_epoch=10.0, initial_epoch=10, validation_steps=None, generator=<keras.pre..., callbacks=[<keras.ca..., epochs=50, validation_data=<keras.pre...)`
  app.launch_new_instance()


Epoch 11/50
10/10 [==============================] - 259s 26s/step - loss: 0.4779 - acc: 0.7500 - val_loss: 0.3997 - val_acc: 0.8512
Epoch 12/50
10/10 [==============================] - 253s 25s/step - loss: 0.4332 - acc: 0.7980 - val_loss: 0.3838 - val_acc: 0.8537
Epoch 13/50
10/10 [==============================] - 263s 26s/step - loss: 0.4465 - acc: 0.7840 - val_loss: 0.3657 - val_acc: 0.8575
Epoch 14/50
10/10 [==============================] - 264s 26s/step - loss: 0.4453 - acc: 0.7920 - val_loss: 0.3554 - val_acc: 0.8600
Epoch 15/50
10/10 [==============================] - 275s 28s/step - loss: 0.4095 - acc: 0.8160 - val_loss: 0.3456 - val_acc: 0.8650
Epoch 16/50
10/10 [==============================] - 271s 27s/step - loss: 0.4050 - acc: 0.8260 - val_loss: 0.3373 - val_acc: 0.8725
Epoch 17/50
10/10 [==============================] - 268s 27s/step - loss: 0.3927 - acc: 0.8260 - val_loss: 0.3248 - val_acc: 0.8688
Epoch 18/50
10/10 [==============================] - 271s 27s/step - 

In [38]:
'''B-5.pred
何個か画像を突っ込んで、予測させてみる
'''

from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions

# 画像データの読み込み
# サイズを150x１５０にリサイズする
filename = './dataset/trainData/dogs/dog0010.jpg'
img = image.load_img(filename, target_size=(150, 150))

# 読み込んだ画像をarrayに変換
img = image.img_to_array(img)

# 3次元テンソル（rows, cols, channels) を
# 4次元テンソル (samples, rows, cols, channels) に変換
# 入力画像は1枚なのでsamples=1でよい
x = np.expand_dims(img, axis=0)
preds = model.predict(preprocess_input(x))

print(x.shape)
print(preds)

# Top-5のクラスを予測する
# VGG16の1000クラスはdecode_predictions()で文字列に変換される
'''
なんかできない。マルチクラスじゃないからかな。

results = decode_predictions(preds, top=1)[0]
for result in results:
    print(result)
'''

(1, 150, 150, 3)
[[ 1.  0.]]


'\nresults = decode_predictions(preds, top=1)[0]\nfor result in results:\n    print(result)\n'